In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

2024-04-14 20:53:42.449678: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-14 20:53:42.597262: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-14 20:53:43.158079: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-14 20:53:44.551878: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
PASTA_EEG = "data/train_eegs/"
PASTA_FOURIER = "data/fourier/"
SPS = 200 #samples per second

frame_size = SPS
half_frame = frame_size//2
hop = (frame_size//2) * 4
frequency_bins = (SPS//2) +1
sampling_rate = SPS

In [63]:
def get_data(eeg):
    return pd.read_parquet(f'data/train_eegs/{eeg}.parquet', engine='pyarrow')


def generate_fourier(df, i, frame_size=frame_size, half_frame=half_frame):
    r_i = []
    for c in df.columns:
        f = np.fft.fft(  df[c][i:i+frame_size] - df[c][i:i+frame_size].mean() )
        r_i.append( np.stack([
                abs(f.real[:half_frame])/half_frame, 
                abs(f.imag[:half_frame])/half_frame
            ], axis=1
        ))

    return np.asarray(r_i)


def get_start_end(row, pos, max_size):
    if pos == len(row['janelas']):
        return max_size, max_size

    j = row['janelas'].split(',')[0].split('-')
    
    return float(j[0])*2, float(j[1])*2


def score_data(row, writer, hop=hop):
    df_data = get_data(row['eeg_id'])
    max_size = df_data.shape[0]
    j_pos = 0
    start, end = get_start_end(row, j_pos, max_size)

    for i in range(0, max_size-hop, hop):
        fourier = generate_fourier(df_data, i)

        if end < i:
            j_pos += 1
            start, end = get_start_end(row, j_pos, max_size)

        if i >= start and i <= end:
            scores = [row['seizure_vote'],row['lpd_vote'],row['gpd_vote'],row['lrda_vote'],row['grda_vote'],row['other_vote'], 0.0 ] 
        else:
            scores = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0] 


        writer.write(
            tf.train.Example(features=tf.train.Features(feature={
                "label": tf.train.Feature(float_list=tf.train.FloatList(value=scores)),
                "data": tf.train.Feature(float_list=tf.train.FloatList(value=fourier.reshape(4000) ))
            })).SerializeToString()
        )



In [10]:
df = pd.read_csv("data/scores.csv")

for i, split in enumerate(np.array_split(df, 10)):
    print(i)
    with tf.io.TFRecordWriter(f"{PASTA_FOURIER}data_{i}") as writer:
        split.apply(lambda x: score_data(x, writer), axis=1)